In [1]:
# %cd D:\IIIT\LAB_SE\ADR\LLM4ADR\0_shot\Decision_0_shot
%cd codes/SE/ADR/LLM4ADR/0_shot/Decision_0_shot

/home2/rudra.dhar/codes/SE/ADR/LLM4ADR/0_shot/Decision_0_shot


In [3]:
pip install tokenizers==0.13.3


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from evaluate import load
import pandas as pd
import nltk
from BARTScore.bart_score import BARTScorer

In [3]:
# CACHE_DIR = '/scratch/adyansh/cache'
# CACHE_DIR = 'D:/IIIT/LAB_SE/ADR/cache'
CACHE_DIR = '/scratch/rudra.dhar/cache'
MODEL_NAME = 'babbage-002'
nltk.data.path.append(CACHE_DIR)

In [4]:
rouge = load('rouge', cache_dir=CACHE_DIR)
bleu = load('bleu', cache_dir=CACHE_DIR)
meteor = load('meteor', cache_dir=CACHE_DIR)
bertscore = load("bertscore", cache_dir=CACHE_DIR)
bartscore = BARTScorer(device='cuda:1', cache_dir=CACHE_DIR)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home2/rudra.dhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home2/rudra.dhar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home2/rudra.dhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
data = pd.read_csv(f'../results/{MODEL_NAME}.csv')
data.rename(columns={'Prediction': 'babbage-002'}, inplace=True)
print(len(data))
data.head()

5297


,Context,Decision,babbage-002
0,## Context\nThe PriveXMl results generated by ...,We increase complexity during insertion by inc...,## Decision Description\nThis decision is to a...
1,## Context\nWe currently have certain specific...,"Due to limited resources and time constraints,...",## The decision for this record is to use the ...
2,"## Context\nCurrently, we move files around in...",- ETL will have a dedicated database tables wh...,## We will move files from Archive folders int...
3,## Context\nMonolith will undergo segmentation...,Create a specific services (ETL and CDH) where...,## We have decided to move the client data hub...
4,## Context\nWe wish to segment into separate p...,We have chosen an event based communication wh...,## 1. The ETL service must be able to communic...


In [6]:
# remove rows in df where MODEL_NAME column is empty
data = data.dropna(subset=[MODEL_NAME])
len(data)

5254

In [8]:
results = {}

results['rouge'] = rouge.compute(predictions=data[MODEL_NAME],references= data['Decision'])
print('Rouge Done')
results['bleu'] = bleu.compute(predictions=data[MODEL_NAME],references= data['Decision'])
print('Bleu Done')
results['meteor'] = meteor.compute(predictions=data[MODEL_NAME],references= data['Decision'])
print('Meteor Done')
results['bertscore'] = bertscore.compute(predictions=data[MODEL_NAME],references= data['Decision'], lang='en', batch_size = 64)
print('BertScore Done')
results['bartscore_recall'] = bartscore.score(data[MODEL_NAME].tolist(), data['Decision'].tolist(), batch_size = 4)
results['bartscore_precision'] = bartscore.score(data['Decision'].tolist(), data[MODEL_NAME].tolist(), batch_size = 4)

Rouge Done
Bleu Done
Meteor Done
BertScore Done


In [12]:
cols = ['precision', 'recall', 'f1']

for c in cols:
    results['bertscore'][c] = pd.Series(results['bertscore'][c]).mean()
results['bartscore'] = pd.Series(results['bartscore']).mean()
results['bartscore_precision'] = pd.Series(results['bartscore_precision']).mean()
results['bartscore_F1'] = (results['bartscore'] + results['bartscore_precision'])/2

In [13]:
results

{'rouge': {'rouge1': 0.14552068859555445,
  'rouge2': 0.03765137690098861,
  'rougeL': 0.09891330993405978,
  'rougeLsum': 0.09887962912859116},
 'bleu': {'bleu': 0.029482551539743336,
  'precisions': [0.09352219670804474,
   0.0338590764246291,
   0.01832591356196787,
   0.013019815348822277],
  'brevity_penalty': 1.0,
  'length_ratio': 3.408742105330583,
  'translation_length': 3192996,
  'reference_length': 936708},
 'meteor': {'meteor': 0.1417808929772791},
 'bertscore': {'precision': 0.787482220431358,
  'recall': 0.8256527816768287,
  'f1': 0.8054393205208882,
  'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)'},
 'bartscore': -4.245395955207414,
 'bartscore_precision': -3.0990172213799245,
 'bartscore_F1': -3.672206588293669}

In [14]:
# write results to a json file
import json
with open(f'../results/{MODEL_NAME}_score.json', 'w') as f:
    json.dump(results, f)